# <center>  Problema do Caixeiro Viajante </center>


---


## Integrantes
Nome: Isaac Higuchi 
RA: 00306191

Nome: Carlos Eduardo de Oliveira 
RA: 00297792

Nome: Thiago Rodrigues de Jesus Carvalho 
RA: 00297767

Nome: João Pedro Taves Araujo 
RA: 00297753


---



---



In [1]:
# [Instalando bibliotecas]
!pip install geopandas
!pip install osmnx
!pip install acopy

^C
^C


KeyboardInterrupt: 

In [ ]:
# [Importando as bliotecas que serão utilizadas]
import geopandas as gpd
import folium
import osmnx as ox
import pandas as pd
from geopy.distance import geodesic

In [ ]:
# [Lendo o arquivo json das cidades]
gdf = gpd.read_file('/content/geojs-35-mun.json')
gdf

In [ ]:
# [Criando função que nos retorna cidades selecionadas]
def buscar_cidade(cidade):
  mapa = gdf.query(f'name == {cidade}')
  return mapa

# [Lista de cidades da viagem]
cidades = [
           'Osasco',
           'Guarulhos',
           'São Paulo',
           'Jaci',
           'Araçatuba',
           'Teodoro Sampaio',
           'Florínia',
           'Piraju',
           'Batatais',
           'Serra Azul',
           'Monte Mor',
           'Campinas',
           'Boituva',
           'Bauru',
           'Itápolis',
           'Pompéia',
           'Ourinhos',
           'Guarujá',
           'Barra do Turvo',
           'Indiana'
]

buscar_cidade(cidades)

In [ ]:
# [Resetando index]
mapa = mapa.reset_index()
mapa

In [ ]:
mapa['centroid'] = mapa['geometry'].centroid
mapa

In [ ]:
p = (mapa['centroid'][0].y, mapa['centroid'][0].x)
nomes = mapa['name'].to_list()

In [ ]:
# [Plotando mapa com as cidades]
m = folium.Map(location=p)

for i, nome in enumerate(nomes):
    p = (mapa['centroid'][i].y, mapa['centroid'][i].x)
    folium.Marker(p, popup=nome).add_to(m)
m

## CALCULANDO AS DISTÂNCIAS

In [ ]:
origem = []
destino = []
distancias = []
for i, nome1 in enumerate(nomes):
    p1 = (mapa['centroid'][i].y, mapa['centroid'][i].x)
    for j, nome2 in enumerate(nomes):    
        p2 = (mapa['centroid'][j].y, mapa['centroid'][j].x)
        origem.append(nome1)
        destino.append(nome2)
        distancias.append( geodesic(p1,p2).km )

In [ ]:
df = pd.DataFrame({'origem': origem, 'destino': destino, 'distancia': distancias})
df

## PROCURANDO A MELHOR ROTA

In [ ]:
import acopy
import networkx as nx
from networkx.algorithms import approximation as approx
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# [Configuração do solver]
# [rho: taxa de evaporação]
# [q: quantidade de ferômonio excretada por uma  formiga a cada iteração]
solver = acopy.Solver(rho=0.03, q=1)

# [alpha: influência do  feromônio e da informação heurística]
# [beta: influência da informação heurística (1/dij)]
colony = acopy.Colony(alpha=1, beta=3) 

In [ ]:
# [Passando os valores numa tupla para usar a biblioteca networkx para fazer o graph]
tupla_valores = [tuple(x) for x in df.values]
tupla_valores

In [ ]:
# Gerar o grafo a partir da matriz de distâncias
g = nx.DiGraph()
g.add_weighted_edges_from(tupla_valores)

# [Solucionar]
# [gen_size: quantidades de formigas (em branco é igual à quantidade de nós)]
# [limit: quantidade de iterações para parada] 
tour = solver.solve(g, colony, limit = 200) 

In [ ]:
tour.nodes

In [ ]:
# [Rota ótima]
tour_resultado = approx.greedy_tsp(g, source='São Paulo')
custo = sum(g[n][nbr]["weight"] for n, nbr in nx.utils.pairwise(tour_resultado))
tour_resultado

In [ ]:
# [Distância total em kilômetros]
print(f'Distância total percorrida foi de {custo:.2f} km.')

### VISUALIZAÇÃO EM MAPA DA ROTA ÓTIMA

In [ ]:
# [Criando função para coletar as coordenadas das cidades da rota ótima]
def get_coord(cidade):
    for i in range(len(tour_resultado)):
      centroid = mapa.query(f'name == "{cidade}"')['centroid']
      coordenada = (float(centroid.y), float(centroid.x))
    return coordenada

In [ ]:
# [Verificando a estrutura do output da função get_coord]
get_coord('Campinas')

In [ ]:
# [Fazendo um loop para pegar a coordenada de todas as cidades]
coordenadas = []
for x in range(len(tour_resultado)):
  coords = get_coord(tour_resultado[x])
  coordenadas.append(coords)

coordenadas

In [ ]:
# [Criando um DataFrame da rota ótima com suas respectivas coordenadas]
df_map = pd.DataFrame(data = {'rota':tour_resultado, 'coordenada': coordenadas})
df_map

In [ ]:
m_resultado = folium.Map()

for c in range(len(df_map)):
    cidade = df_map.coordenada[c]
    folium.Marker(location = cidade, popup = f'{df_map.rota[c]} {c+1}').add_to(m_resultado)
    # folium.Marker(location = cidade, popup = ).add_to(m_resultado)
    folium.PolyLine(locations = [x for x in df_map['coordenada']], 
                line_opacity = 0.5).add_to(m_resultado)
m_resultado

# Integrantes 
Nome: Isaac Higuchi 
RA: 00306191

Nome: Carlos Eduardo de Oliveira 
RA: 00297792

Nome: Thiago Rodrigues de Jesus Carvalho 
RA: 00297767

Nome: João Pedro Taves Araujo 
RA: 00297753